In [ ]:
from pathlib import Path

# 作成したいディレクトリのリスト
directories = [
    "Dataset/nd2",
    "Dataset/train/train_image/augmented_image",
    "Dataset/train/train_image/oliginal_image",
    "Dataset/train/train_image/processed_image",
    "Dataset/train/train_label/augmented_label",
    "Dataset/train/train_label/processed_label",
    "Dataset/val/val_image/oliginal_image",
    "Dataset/val/val_image/processed_image",
    "Dataset/val/val_label/oliginal_label",
    "Dataset/val/val_label/processed_label",
]

# ディレクトリを作成する
for directory in directories:
    path = Path(directory)
    path.mkdir(parents=True, exist_ok=True)
    print(f"ディレクトリ {directory} を作成しました。")


In [ ]:
import nd2 
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import cv2 
import PySimpleGUI as sg
import os

#初期化
def delete_files_in_directories(directories):
    for directory in directories:
        if os.path.exists(directory):
            # ディレクトリ内のすべてのアイテムを取得
            for filename in os.listdir(directory):
                file_path = os.path.join(directory, filename)
                
                # ファイルかどうかを確認
                if os.path.isfile(file_path):
                    try:
                        # ファイルを削除
                        os.remove(file_path)
                        print(f"削除されたファイル: {file_path}")
                    except Exception as e:
                        print(f"ファイル削除中にエラーが発生しました: {e}")
                else:
                    print(f"ディレクトリは削除しません: {file_path}")
        else:
            print(f"ディレクトリが存在しません: {directory}")


def get_image_list(nd2_file):
    """nd2fileから画像のリストを取得"""
    image_list = []
    images = nd2.imread(nd2_file)
    for i, image in enumerate(images):
        image_list.append(image.T)
    return image_list

def convert_nparray(image_list):
    """画像のリストnumpy形式に変換"""
    np_array_list = []
    for i in range(len(image_list)):
        np_img = np.array(image_list[i], dtype=np.uint16)
        np_array_list.append(np_img)
    return np_array_list

def save_image(np_array_list, save_dir):
    """numpyのリストを取得し16bitのpng画像を取得する"""
    for i in range(len(np_array_list)):
        # 12bit → 16bitにスケーリング（各チャネルごとにスケーリング）
        image_16bit = (np_array_list[i].astype(np.uint32) * 16)  # 12bit -> 16bit

        # スケーリング後の値が16bitの範囲（0〜65535）を超えないように制限
        image_16bit = np.clip(image_16bit, 0, 65535)
        # uint16型に変換
        image_16bit = image_16bit.astype(np.uint16)

        # PNG形式で16bit画像を保存
        cv2.imwrite(f'{save_dir}/image_16bit_rgb{i}.png', image_16bit)

directories = [
    "Dataset/train/train_image/augmented_image",
    "Dataset/train/train_image/oliginal_image",
    "Dataset/train/train_image/processed_image",
    "Dataset/train/train_label/augmented_label",
    "Dataset/train/train_label/processed_label",
    "Dataset/val/val_image/oliginal_image",
    "Dataset/val/val_image/processed_image",
    "Dataset/val/val_label/oliginal_label",
    "Dataset/val/val_label/processed_label",
]
delete_files_in_directories(directories)

def main():
    # PySimpleGUIレイアウト定義
    layout = [
        [sg.Text('ND2ファイルを選択:'), sg.Input(key='-ND2_FILE-', enable_events=True), sg.FileBrowse(file_types=(('ND2 Files', '*.nd2'),))],
        [sg.Text('保存先フォルダを選択:'), sg.Input(key='-SAVE_DIR-', enable_events=True), sg.FolderBrowse(initial_folder='Dataset')],
        [sg.Button('実行', key='-RUN-'), sg.Button('終了', key='-EXIT-')],
        [sg.Output(size=(80, 20))]  # 処理ログ表示用
    ]

    # ウィンドウ作成
    window = sg.Window('ND2画像変換ツール', layout)

    while True:
        event, values = window.read()

        # ユーザーがウィンドウを閉じるか「終了」ボタンを押した場合
        if event in (sg.WINDOW_CLOSED, '-EXIT-'):
            break

        # 実行ボタンが押された場合
        if event == '-RUN-':
            nd2_file = values['-ND2_FILE-']
            save_dir = values['-SAVE_DIR-']

            if not nd2_file or not os.path.isfile(nd2_file):
                print('エラー: 有効なND2ファイルを選択してください。')
                continue

            if not save_dir or not os.path.isdir(save_dir):
                print('エラー: 有効な保存先フォルダを選択してください。')
                continue

            try:
                print('画像を処理中...')
                image_list = get_image_list(nd2_file)
                np_array_list = convert_nparray(image_list)
                save_image(np_array_list, save_dir)
                print(f'画像の保存が完了しました: {save_dir}')
            except Exception as e:
                print(f'エラーが発生しました: {e}')

    window.close()

if __name__ == "__main__":
    main()
